## Import các thư viện cần thiết

In [ ]:
import re #regex
import csv
import os
import json
import pandas as pd
from pandas import DataFrame
import urllib.request
import joblib #load, dump pkl
from underthesea import word_tokenize #word_tokenize of lines
import numpy as np
import matplotlib.pyplot as plt
import transformers as ppb # load model BERT
from transformers import BertModel, BertTokenizer
import torch
from sklearn.model_selection import train_test_split

# 1. Data Visualization

## Load data

In [ ]:
file = 'tiki_comments.csv'
df = pd.read_csv(file,sep=",")

### Kích thước của bộ dữ liệu

Dữ liệu có 1399 dòng và 2 cột.

In [ ]:
df.shape

### Ý nghĩa các các cột

Kiểu dữ liệu của các cột

In [ ]:
dtypes = df.dtypes
dtypes

- Cột comment dạng chuỗi là bình luận của người dùng.
- Cột is_trust đánh giá bình luận là tích cực (1) hay tiêu cực (0). 

In [ ]:
pd.set_option('display.max_colwidth',1000)
df.head(10)

## Chuẩn hoá dữ liệu

### Xoá các dòng có giá trị cột comment trùng lặp.

In [ ]:
# Đếm số lượng dòng trùng lặp
num_duplicated_rows = df.duplicated(['comment']).sum()
num_duplicated_rows
# Xoá các dòng trùng
df.drop_duplicates(subset ="comment", keep = 'first', inplace = True) 

print(df.shape)
df.head(10)

### Chuẩn hoá comment
Dữ liệu hiện đang chứa những kí tự như dấu chấm câu, kí tự đặc biệt, icon,... Những kí tự này không cần thiết cho việc khai phá dữ liệu nên ta sẽ tiến hành loại bỏ chúng. Các bước loại bỏ:
- Xoá đi các dấu chấm câu như (. , ? ; ...)
- Xoá đi các kí tự đặc biệt: @,#,*,...
- Xoá khoảng trắng đầu và cuối mỗi comment, xoá  khoảng trắng dư thừa giữa các từ.
- Xoá đi icons và emoji.

In [ ]:
def standardize_data(row):
    row = re.sub(r"[\.,\?]+$-", "", row)
    row = re.sub(r'[^\w]',' ', row) 
    row = row.replace(",", " ").replace(".", " ") \
        .replace(";", " ").replace("“", " ") \
        .replace(":", " ").replace("”", " ") \
        .replace('"', " ").replace("'", " ") \
        .replace("!", " ").replace("?", " ") \
        .replace("-", " ").replace("?", " ")
    row = row.strip()
    row = " ".join(row.split())
    return row

Comment trước khi chuẩn hoá

In [ ]:
df[20:30]

### Sau khi chuẩn hoá

In [ ]:
df['comment'] = df['comment'].apply(standardize_data)
df[20:30]

### Lưu dữ liệu phục vụ cho việc train model

In [ ]:
df.to_csv ('formated__dataset.csv', index = False, header=True)

## Trực quan hoá dữ liệu

<b>Câu hỏi:</b> Tỉ lệ các loại comment trong tập dữ liệu.<br>
Ta sẽ tính toán xem mỗi loại comment chiếm bao nhiêu phầm trăm (%) trong tập dữ liệu.

In [ ]:
#Tính % số lượng các loại comment
num_trust_cmt = df['is_trust'].value_counts()[1]
num_not_trust_cmt = df['is_trust'].value_counts()[0]

trust_ratio = ((num_trust_cmt / df.shape[0])*100).round(2);
not_trust_ratio = ((num_not_trust_cmt / df.shape[0])*100).round(2);
not_trust_ratio

data = [trust_ratio,not_trust_ratio]
labels = ['Tích cực', "Tiêu cực"]

fig = plt.figure(figsize =(10, 7)) 
plt.pie(data, labels = labels, explode = (0.1, 0), autopct='%1.1f%%',shadow=True) 
plt.title('Tỉ lệ % bình luận tích cực và tiêu cực')
plt.axis('equal')
# show plot 
plt.show() 


In [ ]:
#code here

# 2. Predict

## Lấy dữ liệu comment từ  URL

In [ ]:
#code here
def getAllComment(url):
    pass

## Xử lý dữ liệu

### Word tokenizer 

Tách các comment thành từ hoặc cụm từ có nghĩa

In [ ]:
def tokenizer(row):
    return word_tokenize(row, format="text")

In [ ]:
#code here
def processing_data(data):
    # 1. Standardize data
    data_frame = pd.DataFrame(data)
    print('data frame:', data_frame)
    data_frame[0] = data_frame['comment'].apply(standardize_data)

    # 2. Tokenizer
    data_frame[0] = data_frame['comment'].apply(tokenizer)

    # 3. Embedding
    X_val = data_frame['comment']
    return X_val

## Load Pre-train model BERT

In [ ]:
def load_pretrainModel(data):
    '''
    Load pretrain model/ tokenizers
    Return : features
    '''
    model = BertModel.from_pretrained('bert-base-uncased')
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    #encode lines
    tokenized = data.apply((lambda x: tokenizer.encode(x, add_special_tokens = True,max_length=512,truncation=True)))

    # get lenght max of tokenized
    max_len = 0
    for i in tokenized.values:
        if len(i) > max_len:
            max_len = len(i)
    print('max len:', max_len)

    # if lenght of tokenized not equal max_len , so padding value 0
    padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
    print('padded:', padded[1])
    print('len padded:', padded.shape)

    #get attention mask ( 0: not has word, 1: has word)
    attention_mask = np.where(padded ==0, 0,1)
    print('attention mask:', attention_mask[1])

    # Convert input to tensor
    padded = torch.tensor(padded,dtype=torch.long)
    attention_mask = torch.tensor(attention_mask)


    # Load model
    with torch.no_grad():
        last_hidden_states = model(padded, attention_mask =attention_mask)
    #     print('last hidden states:', last_hidden_states)

    features = last_hidden_states[0][:,0,:].numpy()
    print('features:', features)
    
    return features


## Predict

In [ ]:
def predict(url):
    # 1. Load URL and print comments
    data = getAllComment(url)
#     data = load_url_selenium_tiki(url)
    data = processing_data(data)
    features = load_pretrainModel(data)
    # 2. Load weights
    model = joblib.load('save_model.pkl')
    # 3. Result
    result = model.predict(features)
    print(result)
    print(analyze(result))


In [ ]:
predict(url = '')